<a href="https://colab.research.google.com/github/LucasCavalcanti/information-retrieval/blob/master/Expans%C3%A3o_de_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
import time
import math
from tabulate import tabulate

# Realizando a leitura do dataset e tokenização

db = pd.read_csv("https://raw.githubusercontent.com/LucasCavalcanti/ri_lab02/master/results.csv")

documents = db['text']
stopwords = stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


**Questão 1 - Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?** 

As 5 consultas definidas foram:



1.  política
2.   presidente
3.   educação
4.   ministério
5.   empresa

In [0]:
queries = ["política", "presidente", "educação", "ministério", "empresa"]

In [0]:
def buildIndex(documents):
  toker = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
  inverted_index = {}
  n = 0

  for document in documents:
    n += 1
    tokens = toker.tokenize(document.lower())
    for token in set(tokens):
      if (token not in stopwords and len(token) >= 2):
        value = (n, tokens.count(token))
        if (token not in inverted_index.keys()):
          inverted_index[token] = []
          inverted_index[token].append(value)
        else:
          inverted_index[token].append(value)
  return inverted_index
inverted_index = buildIndex(documents)

**Conta a quantidade de documentos em que a consulta e um token aparecem**

In [0]:
def count_nab(index_query, index_item):
  nab = 0
  for item_a in index_query:
    for item_b in index_item:
      if (item_a[0] == item_b[0]):
        nab += 1
  return nab

**Cálculo da métrica Mutual Information**

In [0]:
def mim(na, nb, nab):
  divider = (na * nb)
  if (divider != 0):
    result = nab / divider
    return result
  else:
    return 0.0

**Cálculo da métrica Dice's Coefficient**

In [0]:
def dice(na, nb, nab):
  divider = (na + nb)
  if (divider != 0):
    result = nab / divider
    return result
  else:
    return 0.0

**Cálculo da métrica Expected Mutual Information**

In [0]:
def emim(na, nb, nab, N):
  mim = (nab / (na * nb))
  if (mim != 0):
    result = nab * (math.log(N * mim))
    return result
  else:
    return 0.0

**Cálculo da métrica Chi-Square**

In [0]:
def chi_square(na, nb, nab, N):
  divider = (na * nb)
  if (divider != 0):
    result = math.pow((nab - ((na*nb)/N)),2)/divider
    return result
  else:
    return 0.0

**O método generateMetrics() gera um dicionário onde a chave é consulta e o valor é uma lista com 4 listas, onde cada lista é o top-10 de palavras mais associados a consulta de acordo com cada métrica.**

In [0]:
def generateMetrics():
  all_metrics = {}

  for query in queries:
    if (query in inverted_index.keys()):
      inverted_list_query = inverted_index[query]
      na = len(inverted_list_query)
      mim_list = []
      dice_list = []
      emim_list = []
      chi_square_list = []
      metrics = []

      for item in inverted_index.items():
        if (item[0] != query):
          inverted_list_item = inverted_index[item[0]]
          nb = len(inverted_list_item)
          nab = count_nab(inverted_list_query, inverted_list_item)

          mim_list.append((item[0], mim (na, nb, nab)))
          dice_list.append((item[0], dice (na, nb, nab)))
          emim_list.append((item[0], emim (na, nb, nab, len(documents))))
          chi_square_list.append((item[0], chi_square (na, nb, nab, len(documents))))



      mim_list.sort(key=lambda tup: tup[1])
      dice_list.sort(key=lambda tup: tup[1])
      emim_list.sort(key=lambda tup: tup[1])
      chi_square_list.sort(key=lambda tup: tup[1])
      
      mim_list.reverse()
      dice_list.reverse()
      emim_list.reverse()
      chi_square_list.reverse()
      
      metrics.append(mim_list[:10])
      metrics.append(dice_list[:10])
      metrics.append(emim_list[:10])
      metrics.append(chi_square_list[:10])
      
      all_metrics[query] = metrics


  return all_metrics

In [0]:
all_metrics = generateMetrics()

**O método generateTable() recebe uma consulta como paramêtro e retorna um dataframe com as top-10 palavras mais associadas, de acordo com cada uma das 4 métricas.**

In [0]:
def generateTable(query):
  mim_metrics = []
  dice_metrics = []
  emim_metrics = []
  chi_square_metrics = []

  for item in all_metrics[query][0]:
      mim_metrics.append(item[0])
  
  for item in all_metrics[query][1]:
      dice_metrics.append(item[0])
  
  for item in all_metrics[query][2]:
      emim_metrics.append(item[0])
  
  for item in all_metrics[query][3]:
      chi_square_metrics.append(item[0])


  metrics_df = pd.DataFrame()
  metrics_df['MIM'] = mim_metrics
  metrics_df['Dice'] = dice_metrics
  metrics_df['EMIM'] = emim_metrics
  metrics_df['χ2'] = chi_square_metrics
  return metrics_df

**Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?**

> Para cada consulta gerei a tabela similar a tabela 6.3 do livro texto.


> A métrica que obteve os melhores resultados foi a ***chi-quadrado***, pois foi a métrica que retornou as 10 palavras associadas que mais estão relacionadas com a consulta. A métrica EMIM foi a que mais se aproximou da chi-quadrado, e a que obteve pior resultado foi a MIM.





In [0]:
for query in queries:
  df = generateTable(query)
  print (query)
  print (tabulate(df, headers='keys', tablefmt='psql'))
  print ("")

política
+----+----------------+---------+-----------+--------------+
|    | MIM            | Dice    | EMIM      | χ2           |
|----+----------------+---------+-----------+--------------|
|  0 | governadores   | país    | político  | político     |
|  1 | reparamos      | outros  | cada      | desigualdade |
|  2 | identitária    | vez     | país      | cientista    |
|  3 | vote           | cada    | bolsonaro | eleições     |
|  4 | cuide          | governo | governo   | problemas    |
|  5 | anula          | pode    | outros    | força        |
|  6 | torná-la       | menos   | nacional  | eleição      |
|  7 | aprofundado    | contra  | economia  | economia     |
|  8 | receptividade  | desde   | problemas | democracia   |
|  9 | participativos | parte   | estado    | governos     |
+----+----------------+---------+-----------+--------------+

presidente
+----+-------------------+-----------+-----------+-------------+
|    | MIM               | Dice      | EMIM      | χ2       

**Questão 2 - De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.** 

> Nessa questão utilizei a abordagem documento-por-vez e expandi as consultas com as palavras resultantes da métrica chi-quadrado que foi a que obteve os melhores resultados.


> A medida que aumentamos o número de palavras na expansão da consulta a qualidade dos resultados aumenta, pois o score de documentos que contêm além da palavra da consulta, as novas palavras associadas, aumenta e com isso entram no top-10, retornando assim resultados relevantes para a consulta, que não estavam sendo retornados antes da expansão da consulta.





In [0]:
def documentAtATime(query, inverted_index, k):
  inverted_lists = []
  r = []
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists.append(inverted_index[word])
  for document in range(1, len(documents)+1):
    sd = 0
    for inverted_list in inverted_lists:
      for post in inverted_list:
        if (post[0] == document):
          sd += post[1]
          break
    if (sd != 0):
      heapq.heappush(r, (sd, document))
  return heapq.nlargest(k, r)

**Método para expandir a consulta, onde k é número de palavras que desejo expandir, que no nosso caso será 3, 5 ou 10 e retorna a consulta expandida com as top-k palavras mais associadas, utilizando o resultado da métrica chi-square.**

In [0]:
def queryExpansion(query, k):
  consulta = query
  top_words = generateTable(query).χ2[:k]
  
  for word in top_words:
    consulta += " " + word
  return consulta

**Método que recebe uma consulta e realiza sua expansão com os top-3, top-5 e top-10 palavras mais associadas, faz a consulta utilizando a abordagem documento-por-vez e retorna um dataframe com os resultados dos documentos mais relevantes para cada consulta expandida.**

In [0]:
def generateQueryExpansionTable(query):

  results_top_3 = []
  results_top_5 = []
  results_top_10 = []

  query_expansion = queryExpansion(query, 3)
  query_expansion_documents = documentAtATime(query_expansion, inverted_index, 10)
  for item in query_expansion_documents:
    results_top_3.append(item[1])

  query_expansion = queryExpansion(query, 5)
  query_expansion_documents = documentAtATime(query_expansion, inverted_index, 10)
  for item in query_expansion_documents:
    results_top_5.append(item[1])

  query_expansion = queryExpansion(query, 10)
  query_expansion_documents = documentAtATime(query_expansion, inverted_index, 10)
  for item in query_expansion_documents:
    results_top_10.append(item[1])

  query_expansion_df = pd.DataFrame()
  query_expansion_df['query_expansion_top3'] = results_top_3
  query_expansion_df['query_expansion_top5'] = results_top_5
  query_expansion_df['query_expansion_top10'] = results_top_10
  return query_expansion_df
  

**Gero o dataframe com os resultados do top-10 documentos mais relevantes para cada consulta expandida.**

> **query_expansion_top3** = Consulta expandida com as 3 palavras mais associadas.

> **query_expansion_top5** = Consulta expandida com as 5 palavras mais associadas.

> **query_expansion_top10** = Consulta expandida com as 10 palavras mais associadas.








In [0]:
for query in queries:
  print (query)
  print (tabulate(generateQueryExpansionTable(query), headers='keys', tablefmt='psql'))
  print ("")

política
+----+------------------------+------------------------+-------------------------+
|    |   query_expansion_top3 |   query_expansion_top5 |   query_expansion_top10 |
|----+------------------------+------------------------+-------------------------|
|  0 |                     69 |                     69 |                      69 |
|  1 |                    165 |                     86 |                     248 |
|  2 |                     86 |                    152 |                     166 |
|  3 |                    168 |                    211 |                     165 |
|  4 |                    166 |                    165 |                      86 |
|  5 |                     63 |                     63 |                     210 |
|  6 |                      7 |                    173 |                     138 |
|  7 |                    204 |                    168 |                     211 |
|  8 |                    173 |                    166 |                     1